In [8]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# On 20150607 by MLdP <lopezdeprado@lbl.gov>
# Example for global dynamic optimization

# !python3 -m pip install quandl bokeh
# !conda install -c gurobi gurobi

import sys
import pandas as pd
import numpy as np
import quandl
import gurobipy

from math import sqrt
from bokeh.plotting import figure, show, ColumnDataSource, save
from bokeh.models import Range1d, HoverTool, LabelSet
from bokeh.io import output_notebook, output_file
from gurobipy import *
from itertools import combinations_with_replacement, product

In [62]:
tickers = [
    'AAPL', 'MSFT', 'WMT', 'ACB', 'GE', 'NIO', 'BAC', 'AMZN', 'CMCSA', 'EOAN', 'BMW',
    'IBM', 'RIG', 'CTXS', 'INTC', 'MCD', 'EBAY', 'AXAHF'
]

quandl.ApiConfig.api_key = 'keFUu5WbUqqG7kxqBqEp'
data = quandl.get_table('WIKI/PRICES', ticker=tickers, 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': '2009-01-01', 'lte': '2016-12-31' }, 
                        paginate=True)

# create a new dataframe with 'date' column as index
new = data.set_index('date')

# use pandas pivot function to sort adj_close by tickers
data = new.pivot(columns='ticker')['adj_close']

# check the head of the output
data.tail()


ticker,AAPL,AMZN,BAC,CMCSA,CTXS,EBAY,GE,IBM,INTC,MCD,MSFT,RIG,WMT
date,,,,,,,,,,,,,
2016-12-23,115.080808,760.59,22.346222,34.754708,72.655362,29.79,31.055560,160.477796,35.749710,120.774186,61.864790,14.73,68.153672
2016-12-27,115.811668,771.40,22.356110,34.862932,72.759338,30.24,31.075043,160.891721,35.846409,120.705531,61.903920,15.22,68.310482
2016-12-28,115.317843,772.13,22.079254,34.656323,72.047501,30.01,30.880215,159.977235,35.420932,120.323024,61.620226,14.91,67.928257
2016-12-29,115.288214,765.15,21.752960,34.479229,71.999512,29.98,30.889956,160.371908,35.449942,120.430910,61.532183,14.71,67.879254
2016-12-30,114.389454,749.87,21.851837,34.102904,71.431642,29.69,30.782801,159.784712,35.072815,119.381467,60.788710,14.74,67.742045


In [63]:
GrowthRates = data.pct_change() * 100

syms = GrowthRates.columns
Sigma = GrowthRates.cov()

stats = pd.concat((GrowthRates.mean(), GrowthRates.std()), axis=1)
stats.columns = ['Mean_return', 'Volatility']

extremes = pd.concat((stats.idxmin(), stats.min(), stats.idxmax(), stats.max()), axis=1)
extremes.columns = ['Minimizer', 'Minimum', 'Maximizer', 'Maximum']

print(stats)
print(Sigma)

        Mean_return  Volatility
ticker                         
AAPL       0.128193    1.715613
AMZN       0.154962    2.235206
BAC        0.083595    3.463505
CMCSA      0.087216    1.610116
CTXS       0.088705    2.204196
EBAY       0.101676    2.018386
GE         0.060300    1.791853
IBM        0.049323    1.284423
INTC       0.069249    1.606083
MCD        0.050223    1.023298
MSFT       0.078587    1.593408
RIG       -0.009042    2.879251
WMT        0.025248    1.080554
ticker      AAPL      AMZN        BAC     CMCSA      CTXS      EBAY        GE  \
ticker                                                                          
AAPL    2.943329  1.308661   2.090608  1.050266  1.414407  1.338822  1.215546   
AMZN    1.308661  4.996148   2.091231  1.214684  1.871629  1.901184  1.270969   
BAC     2.090608  2.091231  11.995865  2.582675  2.687423  2.825959  3.742630   
CMCSA   1.050266  1.214684   2.582675  2.592475  1.382330  1.407151  1.536618   
CTXS    1.414407  1.871629   2.687

In [53]:
fig = figure(tools='pan,box_zoom,reset')

source = ColumnDataSource(stats)

fig.add_tools(HoverTool(tooltips=[('Symbol','@ticker'), ('Volatility','@Volatility'), ('Mean return','@Mean_return')]))
fig.circle(x='Volatility', y='Mean_return', source=source)

fig.xaxis.axis_label='Volatility (standard deviation)'
fig.yaxis.axis_label='Mean return'

show(fig)

In [93]:
# Instantiate our model
m = Model("portfolio")

# Create one variable for each stock
portvars = [m.addVar(name=symb, lb=0.0, ub=0.5) for symb in syms]
portvars = pd.Series(portvars, index=syms)
portfolio = pd.DataFrame({'Variables': portvars})

# Commit the changes to the model
m.update()

# The total budget
p_total = portvars.sum()

# The mean return for the portfolio
p_return = stats['Mean_return'].dot(portvars)

# The (squared) volatility of the portfolio
p_risk = Sigma.dot(portvars).dot(portvars)

# Set the objective: minimize risk
m.setObjective(p_risk, GRB.MINIMIZE)

# Fix the budget
m.addConstr(p_total, GRB.EQUAL, 1)

# Select a simplex algorithm (to ensure a vertex solution)
m.setParam('Method', 1)

m.optimize()

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 1 rows, 13 columns and 13 nonzeros
Model has 91 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [5e-01, 5e-01]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 13 columns, 13 nonzeros
Presolved model has 91 quadratic objective terms

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s
      10    6.9232593e-01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  6.923259310e-01


In [94]:
portfolio['Minimum risk'] = portvars.apply(lambda x: x.getAttr('x'))
portfolio

,Variables,Minimum risk
ticker,,
AAPL,<gurobi.Var AAPL (value 0.03881171373530965)>,0.038812
AMZN,<gurobi.Var AMZN (value 0.0)>,0.000000
BAC,<gurobi.Var BAC (value 0.0)>,0.000000
CMCSA,<gurobi.Var CMCSA (value 0.0027899280340250394)>,0.002790
CTXS,<gurobi.Var CTXS (value 0.0)>,0.000000
EBAY,<gurobi.Var EBAY (value 0.0)>,0.000000
GE,<gurobi.Var GE (value 0.0)>,0.000000
IBM,<gurobi.Var IBM (value 0.15558329209402078)>,0.155583
INTC,<gurobi.Var INTC (value 0.0020953265409596896)>,0.002095


In [95]:
# Add the return target
ret50 = 0.5 * extremes.loc['Mean_return','Maximum']
fixreturn = m.addConstr(p_return, GRB.EQUAL, ret50)

m.optimize()

portfolio['50% Max'] = portvars.apply(lambda x: x.getAttr('x'))
portfolio

Optimize a model with 2 rows, 13 columns and 26 nonzeros
Model has 91 quadratic objective terms
Coefficient statistics:
  Matrix range     [9e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [5e-01, 5e-01]
  RHS range        [8e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9232593e-01   0.000000e+00   0.000000e+00      0s
       6    8.9516834e-01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds
Optimal objective  8.951683372e-01


,Variables,Minimum risk,50% Max
ticker,,,
AAPL,<gurobi.Var AAPL (value 0.21093008374609284)>,0.038812,0.210930
AMZN,<gurobi.Var AMZN (value 0.1167127062903465)>,0.000000,0.116713
BAC,<gurobi.Var BAC (value 0.0)>,0.000000,0.000000
CMCSA,<gurobi.Var CMCSA (value 0.08303676691438241)>,0.002790,0.083037
CTXS,<gurobi.Var CTXS (value 0.0)>,0.000000,0.000000
EBAY,<gurobi.Var EBAY (value 0.0008104880366761654)>,0.000000,0.000810
GE,<gurobi.Var GE (value 0.0)>,0.000000,0.000000
IBM,<gurobi.Var IBM (value 0.0220749615081896)>,0.155583,0.022075
INTC,<gurobi.Var INTC (value 0.0)>,0.002095,0.000000


In [96]:
m.setParam('OutputFlag', False)

# Determine the range of returns. Make sure to include the lowest-risk
# portfolio in the list of options
minret = extremes.loc['Mean_return', 'Minimum']
maxret = extremes.loc['Mean_return', 'Maximum']
riskret = extremes.loc['Volatility', 'Minimizer']
riskret = stats.loc[riskret, 'Mean_return']
riskret = sum(portfolio['Minimum risk'] * stats['Mean_return'])
returns = np.unique(np.hstack((np.linspace(minret, maxret, 10000), riskret)))

# Iterate through all returns
risks = returns.copy()
for k in range(len(returns)):
    fixreturn.rhs = returns[k]
    m.optimize()
    risks[k] = sqrt(p_risk.getValue())

In [97]:
fig = figure(tools='pan,box_zoom,reset')

# Individual stocks
fig.add_tools(HoverTool(tooltips=[('Symbol', '@ticker'), ('Volatility', '@Volatility'), ('Mean return', '@Mean_return')]))
fig.circle(x='Volatility', y='Mean_return', source=stats, size=5, color='maroon')

# Divide the efficient frontier into two sections: those with
# a return less than the minimum risk portfolio, those that are greater.
tpos_n = returns >= riskret
tneg_n = returns <= riskret

fig.line(risks[tneg_n], returns[tneg_n], color='red')
fig.line(risks[tpos_n], returns[tpos_n], color='blue')

fig.xaxis.axis_label='Volatility (standard deviation)'
fig.yaxis.axis_label='Mean return'

show(fig) 

In [194]:
def generate_means(size):
    # Generate a random vector of means
    return np.random.normal(size=(size, 1))

def generate_covariance_matrix(size):
    # Generate a random covariance matrix, as <A,A.T> of a random matrix
    rMat = np.random.rand(size, size)
    rCov = np.dot(rMat, rMat.T)
    
    return rCov

def pigeon_hole(k, n):
    # Pigeonhole problem (organize k objects in n slots)
    for j in combinations_with_replacement(range(n), k):
        r = [0] * n
        for i in j:
            r[i] += 1
        yield r
        
def get_all_weights(k, n):
    # 1) Generate partitions
    (parts, w) = (pigeon_hole(k, n), None)

    # 2) Go through partitions
    for part_ in parts:
        w_ = np.array(part_) / float(k)  # abs(weight) vector
        for prod_ in product([-1, 1], repeat=n):  # add sign
            w_signed_ = (w_ * prod_).reshape(-1, 1)
            if w is None:
                w = w_signed_.copy()
            else:
                w = np.append(w, w_signed_, axis=1)
    return w

In [231]:
def evaluate_transaction_costs(capital_allocations, params):
    # Compute t-costs of a particular trajectory
    num_assets = capital_allocations.shape[0]
    num_periods = capital_allocations.shape[1]
    
    transaction_costs = np.zeros(num_periods)
    previous_allocation = np.zeros(num_assets)

    for i in range(num_periods):
        cost = params[i]['c']
        
        transaction_costs[i] = (cost * abs(capital_allocations[:, i] - previous_allocation) ** .5).sum()
        previous_allocation = capital_allocations[:, i].copy()

    return transaction_costs

def evaluate_sharpe_ratio(params, capital_allocations, transaction_costs):
    # Evaluate SR over multiple horizons
    num_periods = capital_allocations.shape[1]
    (mean, cov) = (0, 0)
    
    print(capital_allocations)

    for h in range(num_periods):
        params_ = params[h]
        capital_allocation = capital_allocations[:, h]
        
        mean += np.dot(params_['mean'].T, capital_allocation)[0] - transaction_costs[h]
        cov += np.dot(capital_allocation.T, np.dot(params_['cov'], capital_allocation))

    return mean / (cov ** .5)

def static_optimal_portfolio(covariance, forecasted_mean):
    # Static optimal porftolio
    # Solution to the "unconstrained" portfolio optimization problem
    cov_inv = np.linalg.inv(covariance)

    w = np.dot(cov_inv, forecasted_mean)
    
    # np.dot(w.T, a) == 1
    w /= np.dot(np.dot(forecasted_mean.T, cov_inv), forecasted_mean)

    # re-scale for full investment
    w /= abs(w).sum()

    return w

def dynamic_optimal_portfolio(params, k=None):
    # Dynamic optimal portfolio
    # 1) Generate partitions

    if k is None:
        k = params[0]['mean'].shape[0]

    num_assets = params[0]['mean'].shape[0]
    possible_weights = get_all_weights(k, num_assets)

    sharpe_ratio = None
    best_weights = None

    # 2) Generate trajectories as cartesian products of weights with n repetitions
    for prod_ in product(possible_weights.T, repeat=len(params)):
        # Concatenate product into a trajectory
        weights = np.array(prod_).T
        
        # Compute transaction costs for selected allocations
        transaction_costs = evaluate_transaction_costs(weights, params)
        
        # Evaluate trajectory
        sr_ = evaluate_sharpe_ratio(params, weights, transaction_costs)

        # Store trajectory if better
        if sharpe_ratio is None or sharpe_ratio < sr_:
            sharpe_ratio = sr_
            best_weights = weights.copy()

    return best_weights

def main():

    # 1) Parameters
    (num_assets, horizon) = (4, 1)
    params = []

    for h in range(horizon):
        (mean_, cov_) = (generate_means(num_assets), generate_covariance_matrix(num_assets))
        
        transaction_costs = np.random.uniform(size=cov_.shape[0]) * (np.diag(cov_) ** .5)
        params.append({'mean': mean_, 'cov': cov_, 'c': transaction_costs})

    # 2) Static optimal portfolios
    capital_allocations = None
    for params_ in params:
        weights = static_optimal_portfolio(params_['cov'], params_['mean'])
        if capital_allocations is None:
            capital_allocations = weights.copy()
        else:
            capital_allocations = np.append(capital_allocations, weights, axis=1)

    transaction_costs = evaluate_transaction_costs(capital_allocations, params)
    sharpe_ratio = evaluate_sharpe_ratio(params, capital_allocations, transaction_costs)

    print('static SR:', sharpe_ratio)

    # 3) Dynamic optimal portfolios
    weights = dynamic_optimal_portfolio(params)
    transaction_costs = evaluate_transaction_costs(weights, params)
    sharpe_ratio = evaluate_sharpe_ratio(params, weights, transaction_costs)
    
    print('dynamic SR:', sharpe_ratio)
    return

if __name__ == '__main__':
    main()

[[-0.26113676]
 [ 0.08950251]
 [ 0.45082438]
 [-0.19853634]]
static SR: -7.3948875976619854
[[-1.]
 [-0.]
 [-0.]
 [-0.]]
[[-1.]
 [-0.]
 [-0.]
 [ 0.]]
[[-1.]
 [-0.]
 [ 0.]
 [-0.]]
[[-1.]
 [-0.]
 [ 0.]
 [ 0.]]
[[-1.]
 [ 0.]
 [-0.]
 [-0.]]
[[-1.]
 [ 0.]
 [-0.]
 [ 0.]]
[[-1.]
 [ 0.]
 [ 0.]
 [-0.]]
[[-1.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 1.]
 [-0.]
 [-0.]
 [-0.]]
[[ 1.]
 [-0.]
 [-0.]
 [ 0.]]
[[ 1.]
 [-0.]
 [ 0.]
 [-0.]]
[[ 1.]
 [-0.]
 [ 0.]
 [ 0.]]
[[ 1.]
 [ 0.]
 [-0.]
 [-0.]]
[[ 1.]
 [ 0.]
 [-0.]
 [ 0.]]
[[ 1.]
 [ 0.]
 [ 0.]
 [-0.]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[-0.75]
 [-0.25]
 [-0.  ]
 [-0.  ]]
[[-0.75]
 [-0.25]
 [-0.  ]
 [ 0.  ]]
[[-0.75]
 [-0.25]
 [ 0.  ]
 [-0.  ]]
[[-0.75]
 [-0.25]
 [ 0.  ]
 [ 0.  ]]
[[-0.75]
 [ 0.25]
 [-0.  ]
 [-0.  ]]
[[-0.75]
 [ 0.25]
 [-0.  ]
 [ 0.  ]]
[[-0.75]
 [ 0.25]
 [ 0.  ]
 [-0.  ]]
[[-0.75]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]]
[[ 0.75]
 [-0.25]
 [-0.  ]
 [-0.  ]]
[[ 0.75]
 [-0.25]
 [-0.  ]
 [ 0.  ]]
[[ 0.75]
 [-0.25]
 [ 0.  ]
 [-0.  ]]
[[ 0.75]
 [-0.25]
 [ 0.  ]
 [ 0.  ]]
[[ 0